In [ ]:
import os

print(os.getcwd())  # Prints the current working directory

c:\DataScience\Spring2025\FinalProject_MS\final_project\AI-Driven_L2C_Optimization\Code\Lead Generation and Qualification\Lead Scoring


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
import os
import pandas as pd

# Change working directory to AI-Driven_L2C_Optimization
os.chdir(
    "C:/DataScience/Spring2025/FinalProject_MS/final_project/AI-Driven_L2C_Optimization"
)

# Now, the relative path works
file_path = "Data/CRM_sales_Data/accounts.csv"

df = pd.read_csv(file_path)
print(df.head())

            account     sector  year_established  revenue  employees  \
0  Acme Corporation  technolgy              1996  1100.04       2822   
1        Betasoloin    medical              1999   251.41        495   
2          Betatech    medical              1986   647.18       1185   
3        Bioholding    medical              2012   587.34       1356   
4           Bioplex    medical              1991   326.82       1016   

  office_location subsidiary_of  
0   United States           NaN  
1   United States           NaN  
2           Kenya           NaN  
3      Philipines           NaN  
4   United States           NaN  


In [ ]:
# Load Datasets
accounts = pd.read_csv("Data/CRM_sales_Data/accounts.csv")
products = pd.read_csv("Data/CRM_sales_Data/products.csv")
sales_teams = pd.read_csv("Data/CRM_sales_Data/sales_teams.csv")
sales_pipeline = pd.read_csv("Data/CRM_sales_Data/sales_pipeline.csv")

In [35]:
accounts.isna().sum()

account              0
sector               0
year_established     0
revenue              0
employees            0
office_location      0
subsidiary_of       70
dtype: int64

In [37]:
accounts["subsidiary_of"].fillna("Unknown", inplace=True)
accounts.isna().sum()

account             0
sector              0
year_established    0
revenue             0
employees           0
office_location     0
subsidiary_of       0
dtype: int64

In [39]:
sales_teams.isna().sum()
sales_pipeline.isna().sum()

opportunity_id       0
sales_agent          0
product              0
account           1425
deal_stage           0
engage_date        500
close_date        2089
close_value       2089
dtype: int64

In [42]:
sales_pipeline.isna().sum()

opportunity_id       0
sales_agent          0
product              0
account           1425
deal_stage           0
engage_date        500
close_date        2089
close_value       2089
dtype: int64

In [ ]:
sales_pipeline["account"].fillna("unknow", inplace=True)

In [ ]:
sales_pipeline["engage_date"] = pd.to_datetime(sales_pipeline["engage_date"])
sales_pipeline["close_date"] = pd.to_datetime(sales_pipeline["close_date"])

In [ ]:
sales_pipeline["engage_date"].fillna(
    sales_pipeline["engage_date"].median(), inplace=True
)
sales_pipeline["close_date"].fillna(
    sales_pipeline["close_date"].median(), inplace=True
)
sales_pipeline["close_value"].fillna(
    sales_pipeline["close_value"].median(), inplace=True
)

C:\Users\sadiq\AppData\Local\Temp\ipykernel_34876\2213439418.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sales_pipeline["engage_date"].fillna(sales_pipeline["engage_date"].median(), inplace=True)
C:\Users\sadiq\AppData\Local\Temp\ipykernel_34876\2213439418.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [60]:
sales_pipeline.isna().sum()

opportunity_id    0
sales_agent       0
product           0
account           0
deal_stage        0
engage_date       0
close_date        0
close_value       0
dtype: int64

In [ ]:
# Merge Datasets on Relevant Keys
sales_data = sales_pipeline.merge(accounts, on="account", how="left")
sales_data = sales_data.merge(products, on="product", how="left")
sales_data = sales_data.merge(sales_teams, on="sales_agent", how="left")
# print(sales_data)
print(sales_data.isna().sum())
sales_data.head(20)

opportunity_id         0
sales_agent            0
product                0
account                0
deal_stage             0
engage_date            0
close_date             0
close_value            0
sector              1425
year_established    1425
revenue             1425
employees           1425
office_location     1425
subsidiary_of       1425
series              1480
sales_price         1480
manager                0
regional_office        0
dtype: int64


,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,sector,year_established,revenue,employees,office_location,subsidiary_of,series,sales_price,manager,regional_office
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0,retail,2001.0,718.62,2448.0,United States,Unknown,GTX,1096.0,Dustin Brinkmann,Central
1,Z063OYW0,Darcel Schlecht,GTXPro,Isdom,Won,2016-10-25,2017-03-11,4514.0,medical,2002.0,3178.24,4540.0,United States,Unknown,NaN,NaN,Melvin Marxen,Central
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0,retail,2001.0,718.62,2448.0,United States,Unknown,MG,55.0,Melvin Marxen,Central
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0,software,1998.0,2714.90,2641.0,United States,Acme Corporation,GTX,550.0,Dustin Brinkmann,Central
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0,services,1982.0,792.46,1299.0,United States,Unknown,GTX,550.0,Summer Sewald,West
5,ZNBS69V1,Anna Snelling,MG Special,Ron-tech,Won,2016-10-29,2017-03-01,49.0,medical,1992.0,3922.42,6837.0,United States,Unknown,MG,55.0,Dustin Brinkmann,Central
6,9ME3374G,Vicki Laflamme,MG Special,J-Texon,Won,2016-10-30,2017-03-02,57.0,retail,1989.0,1388.67,3583.0,United States,Unknown,MG,55.0,Celia Rouche,West
7,7GN8Q4LL,Markita Hansen,GTX Basic,Cheers,Won,2016-11-01,2017-03-07,601.0,entertainment,1993.0,4269.90,6472.0,United States,Massive Dynamic,GTX,550.0,Celia Rouche,West
8,OLK9LKZB,Niesha Huffines,GTX Plus Basic,Zumgoity,Won,2016-11-01,2017-03-03,1026.0,medical,1984.0,441.08,1210.0,United States,Unknown,GTX,1096.0,Melvin Marxen,Central
9,HAXMC4IX,James Ascencio,MG Advanced,unknow,Engaging,2016-11-03,2017-08-02,472.0,NaN,NaN,NaN,NaN,NaN,NaN,MG,3393.0,Summer Sewald,West


In [ ]:
# Fill categorical columns with "Unknown"
sales_data[
    [
        "sector",
        "year_established",
        "office_location",
        "subsidiary_of",
        "series",
    ]
] = sales_data[
    [
        "sector",
        "year_established",
        "office_location",
        "subsidiary_of",
        "series",
    ]
].fillna(
    "Unknown"
)

# Fill numerical columns with their median values
sales_data[["revenue", "employees", "sales_price"]] = sales_data[
    ["revenue", "employees", "sales_price"]
].fillna(sales_data[["revenue", "employees", "sales_price"]].median())
sales_data.isna().sum()

opportunity_id      0
sales_agent         0
product             0
account             0
deal_stage          0
engage_date         0
close_date          0
close_value         0
sector              0
year_established    0
revenue             0
employees           0
office_location     0
subsidiary_of       0
series              0
sales_price         0
manager             0
regional_office     0
dtype: int64

In [ ]:
# Feature Engineering: Convert Deal Stage to Binary Outcome (1 = Won, 0 = Lost)
sales_data["deal_won"] = sales_data["deal_stage"].apply(
    lambda x: 1 if x == "Won" else 0
)
print(sales_data)

     opportunity_id        sales_agent         product  account   deal_stage  \
0          1C1I7A6R        Moses Frase  GTX Plus Basic  Cancity          Won   
1          Z063OYW0    Darcel Schlecht          GTXPro    Isdom          Won   
2          EC4QE1BX    Darcel Schlecht      MG Special  Cancity          Won   
3          MV1LWRNH        Moses Frase       GTX Basic  Codehow          Won   
4          PE84CX4O          Zane Levy       GTX Basic   Hatfan          Won   
...             ...                ...             ...      ...          ...   
8795       9MIWFW5J  Versie Hillebrand     MG Advanced   unknow  Prospecting   
8796       6SLKZ8FI  Versie Hillebrand     MG Advanced   unknow  Prospecting   
8797       LIB4KUZJ  Versie Hillebrand     MG Advanced   unknow  Prospecting   
8798       18IUIUK0  Versie Hillebrand     MG Advanced   unknow  Prospecting   
8799       8I5ONXJX  Versie Hillebrand     MG Advanced   unknow  Prospecting   

     engage_date close_date  close_valu

In [ ]:
# Encode Categorical Features
encoder = LabelEncoder()
categorical_cols = [
    "sector",
    "office_location",
    "subsidiary_of",
    "series",
    "sales_agent",
    "manager",
    "regional_office",
]

for col in categorical_cols:
    sales_data[col] = encoder.fit_transform(sales_data[col].astype(str))
    # print(sales_data[col])

In [ ]:
# Select Features for Training
features = [
    "sector",
    "year_established",
    "revenue",
    "employees",
    "office_location",
    "subsidiary_of",
    "series",
    "sales_price",
    "sales_agent",
    "manager",
    "regional_office",
    "close_value",
]
X = sales_data[features]
y = sales_data["deal_won"]

In [ ]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
if not isinstance(X_train, pd.DataFrame):
    X_train = pd.DataFrame(X_train)

if not isinstance(X_test, pd.DataFrame):
    X_test = pd.DataFrame(X_test)

X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

In [ ]:
categorical_columns = X_train.select_dtypes(include=["object"]).columns
X_train = pd.get_dummies(X_train, columns=categorical_columns, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_columns, drop_first=True)

In [ ]:
X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Normalize Data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Dictionary to Store Models and Their Results
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    # "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "SVM": SVC(probability=True),  # Enables probability predictions
}

# Train and Evaluate Each Model
best_model = None
best_accuracy = 0

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd  # Ensure pandas is imported

# Replace 'Unknown' with NaN
X_train = pd.DataFrame(X_train)  # Ensure X_train is a DataFrame
X_test = pd.DataFrame(X_test)

X_train.replace("Unknown", np.nan, inplace=True)
X_test.replace("Unknown", np.nan, inplace=True)

# Handle missing values
imputer = SimpleImputer(strategy="most_frequent")
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Convert categorical columns
encoder = LabelEncoder()
for col in X_train.select_dtypes(include=["object"]).columns:
    X_train[col] = encoder.fit_transform(X_train[col].astype(str))
    X_test[col] = encoder.transform(X_test[col].astype(str))

# Apply StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Ensure y_train is numeric (no need to re-encode if already 0s and 1s)
# y_train and y_test are already binary (0/1), so this step is unnecessary
# If y_train is categorical, ensure it's numeric
if not np.issubdtype(y_train.dtype, np.number):
    y_train = LabelEncoder().fit_transform(y_train)
    y_test = LabelEncoder().transform(y_test)

In [ ]:
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"{model_name} Accuracy: {accuracy:.2f}")
    print(classification_report(y_test, y_pred))
    # Save Best Model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model_name

print(f"\nBest Model: {best_model} with Accuracy: {best_accuracy:.2f}")


Training Logistic Regression...
Logistic Regression Accuracy: 0.90
              precision    recall  f1-score   support

           0       0.88      0.93      0.91       894
           1       0.92      0.87      0.90       866

    accuracy                           0.90      1760
   macro avg       0.90      0.90      0.90      1760
weighted avg       0.90      0.90      0.90      1760


Training Random Forest...
Random Forest Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       894
           1       1.00      1.00      1.00       866

    accuracy                           1.00      1760
   macro avg       1.00      1.00      1.00      1760
weighted avg       1.00      1.00      1.00      1760


Training SVM...
SVM Accuracy: 0.92
              precision    recall  f1-score   support

           0       0.98      0.85      0.91       894
           1       0.86      0.98      0.92       866

    accuracy          

In [ ]:
# Store column names and indexes before transforming
X_train_columns = X_train.shape[1]  # Save the number of columns
X_test_columns = X_test.shape[1]  # Save the number of columns

# Reshape X_train and X_test if needed
X_train = X_train.reshape(-1, X_train_columns)
X_test = X_test.reshape(-1, X_test_columns)

# Convert back to DataFrame after scaling
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

# Assign Lead Scores from the Best Model
lead_scores = models[best_model].predict_proba(X_test)[:, 1] * 100

# Fix Index Issue: Restore Original Index
X_test.index = y_test.index  # Ensure X_test has the same index as y_test

# Ensure index mapping by using sales_data.loc[]
sales_data.loc[X_test.index, "lead_score"] = lead_scores  # Assign lead scores

# Save Scored Leads
sales_data[["account", "sales_agent", "product", "lead_score"]].to_csv(
    "lead_scores.csv", index=False
)
print("Lead scores saved successfully!")

Lead scores saved successfully!
